In [13]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader
from tqdm import tqdm
import bisect
import json
from datetime import date

In [3]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [4]:
perf_path = replace_slash('input_data\\' + 'armitz/miner/ARX_PERF.xlsx')
fes_path = replace_slash('input_data\\armitz/miner/QINF.xlsx')

In [5]:
perf_ints, perf_df = perf_reader(perf_path)
fes_df = fes_reader(fes_path)
SOIL_CUT = 60

started reading perf xl
done reading perf xl
started transform perf table to dict
done
started reading fes xl
done reading fes xl


In [6]:
fes_df

,well,top,bot,soil
0,1,NaN,1648.4,NaN
1,1,1669.6,1670.6,69.5
2,1,1678.4,1679.2,NaN
3,1,1683.0,1684.0,NaN
4,1,1686.0,1687.0,NaN
...,...,...,...,...
156269,32960,1901.0,1901.8,52.9
156270,32960,1903.0,1905.0,NaN
156271,32960,1914.0,1915.4,NaN
156272,32960,1917.6,1919.0,NaN


In [16]:
def get_actual_perf(perf_df):
    act_perf = []
    for well in tqdm(perf_df['well'].unique()):
        well_df = perf_df[perf_df['well'] == well]
        act_perf_well = []
        for date in well_df['date'].unique():
            date_df = well_df[well_df['date'] == date]
            for top, bot, perf_type in date_df[['top', 'bot', 'type']].values:
                idx = bisect.bisect_left([t['bot'] for t in act_perf_well], top)
                if idx == len(act_perf_well):
                    act_perf_well.append({'well': well, 'top': top,'bot': bot, 'perf_type': perf_type})
                else:
                    if act_perf_well[idx]['top'] > top:
                        if act_perf_well[idx]['perf_type'] == perf_type:
                            act_perf_well[idx]['top'] = top
                        else:
                            act_perf_well.insert(idx, {'well': well, 'top': top,'bot': bot if act_perf_well[idx]['top'] > bot else act_perf_well[idx]['top'],
                                                   'perf_type': perf_type})
                    if act_perf_well[idx]['bot'] < bot:
                        if act_perf_well[idx]['perf_type'] == perf_type:
                            act_perf_well[idx]['bot'] = bot
                        else:
                            act_perf_well.insert(idx + 1, {'well': well, 'top': act_perf_well[idx]['bot'],'bot': bot, 'perf_type': perf_type})

#         act_perf[well] = act_perf_well
        act_perf.extend(act_perf_well)
    return act_perf


#

perf =  get_actual_perf(perf_df)

100%|██████████| 4093/4093 [00:45<00:00, 89.64it/s] 


In [6]:
perf_df

,well,type,date,top,bot
48492,9642д,1,1991-03-11 00:00:00,1316.0,1340.5
48491,9642д,0,1991-03-10 00:00:00,1676.8,1679.8
48490,9642д,1,1982-01-16 00:00:00,1676.8,1679.8
54866,9636,1,2003-09-10 23:00:00,1736.0,1738.0
54867,9636,1,2003-09-10 23:00:00,1739.7,1740.9
...,...,...,...,...,...
24139,12,1,1991-04-18 23:00:00,10.5,36.0
24131,11,1,1991-04-11 23:00:00,67.5,69.0
24132,11,1,1991-04-11 23:00:00,69.0,70.5
24130,9,0,2007-09-28 23:00:00,37.0,42.0


In [8]:
perf_df['date'] = perf_df['date'].dt.date

In [11]:
perf_df.loc[0, 'date'].year

2002

In [25]:

datetime2 = date(year=2002, month=1, day=1)

In [26]:
perf_df.drop(perf_df[perf_df['date'] > datetime2].index)

,well,type,date,top,bot
48492,9642д,1,1991-03-11,1316.0,1340.5
48491,9642д,0,1991-03-10,1676.8,1679.8
48490,9642д,1,1982-01-16,1676.8,1679.8
54864,9636,1,2001-09-18,1736.2,1738.1
54865,9636,1,2001-09-18,1739.7,1740.9
...,...,...,...,...,...
24140,13,1,1991-05-07,45.0,65.0
24139,12,1,1991-04-18,10.5,36.0
24131,11,1,1991-04-11,67.5,69.0
24132,11,1,1991-04-11,69.0,70.5
